In [1]:
import re
import json
import time as time
import numpy as np
import pandas as pd
from functools import partial
from datetime import date, timedelta

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import plotly.figure_factory as ff
import plotly.express as px
import plotly.offline as pyo

## Own specific functions 
from functions import *

In [2]:
def calculate_years(days):
    """
    Method used to calculate years based on date (today - quantity of days).

    Parameters:
    -----------------
        days (int): Numbers of day to rest of today

    Returns:
    -----------------
        years (int): Numbers of years
    """

    today = date.today()
    initial_date = today - timedelta(abs(days))
    years = today.year - initial_date.year - ((today.month, today.day) < (initial_date.month, initial_date.day))

    return years

In [3]:
# Columns to read on CSVs
COLUMNS = [
    "SK_ID_CURR", "CODE_GENDER", "DAYS_BIRTH", "DAYS_EMPLOYED",
    "CNT_CHILDREN", "FLAG_OWN_REALTY", "FLAG_OWN_CAR",
    "AMT_INCOME_TOTAL", "AMT_CREDIT", "TARGET"
]

In [4]:
# Reading the csv
df_clients_to_predict = pd.read_csv("datasets/df_customers_to_predict.csv")

In [5]:
df_analysis(df_clients_to_predict, "df_clients_to_predict", analysis_type="header")


Analysis Header of df_clients_to_predict dataset
--------------------------------------------------------------------------------
- Dataset shape:			 48744 rows and 796 columns
- Total of NaN values:			 0
- Percentage of NaN:			 0.0 %
- Total of infinite values:		 0
- Percentage of infinite values:	 0.0 %
- Total of full duplicates rows:	 0
- Total of empty rows:			 0
- Total of empty columns:		 0
- Unique indexes:			 True
- Memory usage:				 296.0 MB


In [6]:
# Reading the csv
df_optimized = pd.read_csv("datasets/df_optimized.csv", usecols=COLUMNS, low_memory=True)

In [7]:
df_analysis(df_optimized, "df_optimized", analysis_type="header")


Analysis Header of df_optimized dataset
--------------------------------------------------------------------------------
- Dataset shape:			 356251 rows and 10 columns
- Total of NaN values:			 48744
- Percentage of NaN:			 1.37 %
- Total of infinite values:		 0
- Percentage of infinite values:	 0.0 %
- Total of full duplicates rows:	 0
- Total of empty rows:			 0
- Total of empty columns:		 0
- Unique indexes:			 True
- Memory usage:				 27.2 MB


In [8]:
xx

NameError: name 'xx' is not defined

In [ ]:
df_optimized.to_csv("datasets\df_optimized_and_reduced.csv", index=False)

In [ ]:
df_optimized["AGE"] = df_optimized["DAYS_BIRTH"].apply(lambda x: calculate_years(x))

In [ ]:
df_optimized.head()

In [ ]:
df_customer_selected = df_clients_to_predict[df_clients_to_predict["SK_ID_CURR"]==100013]
df_customer_selected["AGE"] = df_customer_selected["DAYS_BIRTH"].apply(lambda x: calculate_years(x))
df_customer_selected

In [ ]:
df_customer_selected["AGE"].values[0]

In [ ]:
df_optimized_by_target_repaid = df_optimized[df_optimized["TARGET"] == 0]
df_optimized_by_target_not_repaid = df_optimized[df_optimized["TARGET"] == 1]

In [ ]:
ages_data_repaid = df_optimized_by_target_repaid.groupby("AGE").size()
ages_data_repaid = pd.DataFrame(ages_data_repaid).reset_index()
ages_data_repaid.columns = ["AGE", "AMOUNT"]
ages_data_repaid = ages_data_repaid.set_index("AGE").to_dict()["AMOUNT"]


ages_data_not_repaid = df_optimized_by_target_not_repaid.groupby("AGE").size()
ages_data_not_repaid = pd.DataFrame(ages_data_not_repaid).reset_index()
ages_data_not_repaid.columns = ["AGE", "AMOUNT"]
ages_data_not_repaid = ages_data_not_repaid.set_index("AGE").to_dict()["AMOUNT"]

In [ ]:
ages_data_repaid_list = [key for key, val in ages_data_repaid.items() for _ in range(val)]
ages_data_not_repaid_list = [key for key, val in ages_data_not_repaid.items() for _ in range(val)]

In [ ]:
config = {
    "displayModeBar": False,
    "displaylogo": False
}

group_labels = ["Repaid", "Not repaid"]

fig_ages = ff.create_distplot([ages_data_repaid_list, ages_data_not_repaid_list], 
                              group_labels, show_hist=False, show_rug=False, 
                             colors=["Green", "Red"])
fig_ages.update_layout(
    paper_bgcolor="white",
    font={
        "family": "sans serif"
    },
    autosize=False,
    width=500,
    height=360,
    margin=dict(
        l=50, r=50, b=0, t=20, pad=0
    ),
    title={
        "text" : "Client age vs Current clients",
        "y" : 1,
        "x" : 0.45,
        "xanchor" : "center",
        "yanchor" : "top"
    },
    xaxis_title="Ages",
    yaxis_title="Density",
    legend={
        "traceorder" : "normal"
    }
)
fig_ages.add_vline(x=df_customer_selected["AGE"].values[0], line_width=3,
                   line_dash="dash", line_color="green", annotation_text="client's age")
pyo.plot(fig_ages, config=config, filename="example2.html")

In [ ]:
df_optimized["YEARS_WORK"] = df_optimized["DAYS_EMPLOYED"].apply(lambda x: calculate_years(x))

In [ ]:
df_optimized.head()

In [ ]:
df_customer_selected = df_clients_to_predict[df_clients_to_predict["SK_ID_CURR"]==100013]
df_customer_selected["YEARS_WORK"] = df_customer_selected["DAYS_EMPLOYED"].apply(lambda x: calculate_years(x))
df_customer_selected["YEARS_WORK"].values[0]

In [ ]:
df_optimized_by_target_repaid = df_optimized[df_optimized["TARGET"] == 0]
df_optimized_by_target_not_repaid = df_optimized[df_optimized["TARGET"] == 1]

In [ ]:
ages_data_repaid = df_optimized_by_target_repaid.groupby("YEARS_WORK").size()
ages_data_repaid = pd.DataFrame(ages_data_repaid).reset_index()
ages_data_repaid.columns = ["YEARS_WORK", "AMOUNT"]
ages_data_repaid = ages_data_repaid.set_index("YEARS_WORK").to_dict()["AMOUNT"]


ages_data_not_repaid = df_optimized_by_target_not_repaid.groupby("YEARS_WORK").size()
ages_data_not_repaid = pd.DataFrame(ages_data_not_repaid).reset_index()
ages_data_not_repaid.columns = ["YEARS_WORK", "AMOUNT"]
ages_data_not_repaid = ages_data_not_repaid.set_index("YEARS_WORK").to_dict()["AMOUNT"]

In [ ]:
ages_data_repaid_list = [key for key, val in ages_data_repaid.items() for _ in range(val)]
ages_data_not_repaid_list = [key for key, val in ages_data_not_repaid.items() for _ in range(val)]

In [ ]:
config = {
    "displayModeBar": False,
    "displaylogo": False
}

group_labels = ["Repaid", "Not repaid"]

fig_ages = ff.create_distplot([ages_data_repaid_list, ages_data_not_repaid_list], 
                              group_labels, show_hist=False, show_rug=False, 
                             colors=["Green", "Red"])
fig_ages.update_layout(
    paper_bgcolor="white",
    font={
        "family": "sans serif"
    },
    autosize=False,
    width=500,
    height=360,
    margin=dict(
        l=50, r=50, b=0, t=20, pad=0
    ),
    title={
        "text" : "Client years worked vs Current clients",
        "y" : 1,
        "x" : 0.45,
        "xanchor" : "center",
        "yanchor" : "top"
    },
    xaxis_title="Years worked",
    yaxis_title="Density",
    legend={
        "traceorder" : "normal"
    }
)
fig_ages.add_vline(x=df_customer_selected["YEARS_WORK"].values[0], line_width=3,
                   line_dash="dash", line_color="green", annotation_text="years worked by the client")
pyo.plot(fig_ages, config=config, filename="example3.html")